# Exploration GMAIL

## Auth and connect to gmail

In [2]:
from pathlib import Path
import json

PROJECT_ROOT = Path.cwd()
while not (PROJECT_ROOT / "pyproject.toml").exists():
    PROJECT_ROOT = PROJECT_ROOT.parent

CLIENT_SECRET_PATH = PROJECT_ROOT / "secrets/google_client_secret.json"
TOKEN_PATH = PROJECT_ROOT / "secrets/gmail_token.json"

In [ ]:
from hiring_compass_au.ingestion.gmail.auth_and_build import authenticate_gmail, build_gmail_service

creds = authenticate_gmail(
    client_secret_path=CLIENT_SECRET_PATH,
    token_path=TOKEN_PATH,
)

service = build_gmail_service(creds)
print("OK: service built")

OK: service built


## Get email list

In [4]:
service.users().messages().list

<bound method createMethod.<locals>.method of <googleapiclient.discovery.Resource object at 0x124685550>>

In [5]:
response = service.users().messages().list(
    userId="me",
    q="from:jobmail@s.seek.com.au"
).execute()

messages = response.get("messages", [])
len(messages), messages[:3]


(65,
 [{'id': '19c2acfa924a9516', 'threadId': '19c2acfa924a9516'},
  {'id': '19c2a83d45fad2f5', 'threadId': '19c2a83d45fad2f5'},
  {'id': '19c2a800505bda07', 'threadId': '19c2a800505bda07'}])

## Learn to explore an email

### Identifying

In [6]:
message = service.users().messages().get(
    userId="me",
    id=messages[0]["id"],
    format="full",
    ).execute()

message

{'id': '19c2acfa924a9516',
 'threadId': '19c2acfa924a9516',
 'labelIds': ['CATEGORY_UPDATES', 'INBOX'],
 'snippet': 'Preacta Recruitment, oOh!, Talenza and other companies have new roles for you. seek Hi Hiring, Based on your saved search for machine learning engineer, we&#39;ve found 20 new jobs that could be right',
 'payload': {'partId': '',
  'mimeType': 'multipart/alternative',
  'filename': '',
  'headers': [{'name': 'Delivered-To', 'value': 'hiringcompassau@gmail.com'},
   {'name': 'Received',
    'value': 'by 2002:a05:693c:2d86:b0:2b0:6b2e:6320 with SMTP id rt6csp3790417dyc;        Wed, 4 Feb 2026 14:39:38 -0800 (PST)'},
   {'name': 'X-Received',
    'value': 'by 2002:ad4:5c44:0:b0:894:68f7:40f2 with SMTP id 6a1803df08f44-89522122793mr65461626d6.2.1770244778332;        Wed, 04 Feb 2026 14:39:38 -0800 (PST)'},
   {'name': 'ARC-Seal',
    'value': 'i=1; a=rsa-sha256; t=1770244778; cv=none;        d=google.com; s=arc-20240605;        b=lECLpJsh0BF813nGcidGsY4rFKZQxq0eLxXOll+mm+Q9s

In [9]:
message["payload"]

{'partId': '',
 'mimeType': 'multipart/alternative',
 'filename': '',
 'headers': [{'name': 'Delivered-To', 'value': 'hiringcompassau@gmail.com'},
  {'name': 'Received',
   'value': 'by 2002:a05:693c:2d86:b0:2b0:6b2e:6320 with SMTP id rt6csp3790417dyc;        Wed, 4 Feb 2026 14:39:38 -0800 (PST)'},
  {'name': 'X-Received',
   'value': 'by 2002:ad4:5c44:0:b0:894:68f7:40f2 with SMTP id 6a1803df08f44-89522122793mr65461626d6.2.1770244778332;        Wed, 04 Feb 2026 14:39:38 -0800 (PST)'},
  {'name': 'ARC-Seal',
   'value': 'i=1; a=rsa-sha256; t=1770244778; cv=none;        d=google.com; s=arc-20240605;        b=lECLpJsh0BF813nGcidGsY4rFKZQxq0eLxXOll+mm+Q9sQcBmr5O0egrwB028UbAD/         GEk2qVmFVvz+v9fIT6akSl+qBiZGm52fNcCLN3yzOXORG+JNJpTCzf7N510RbQR7jh90         6CA6qOn3QcvAlhmSFeedBva43VgJKoQH7uoyf62HkuS6EguxY0myVfUMCFm3sHw1gY7D         Z0685Tm997I7CyH4EaM2LrwLlqCvkJntiGq8+l7tPqZFNkfZU2TgukCLeq+OH75cNTW/         W1+pwTpsaZmz+RN8PV1ELhu0aXEC/Ur6y/JZEADQ/ltwVPiQCFJT6YPBoINzGlTqtUaC         6lH

### Get html

In [8]:
import base64

def decode_base64url(data: str) -> str:
    # Gmail envoie du base64url parfois sans padding (=)
    data = data + "=" * (-len(data) % 4)
    return base64.urlsafe_b64decode(data.encode("utf-8")).decode("utf-8", errors="replace")

def walk_parts(payload: dict):
    # DFS sur la structure MIME
    stack = [payload]
    while stack:
        part = stack.pop()
        yield part
        for p in part.get("parts", []) or []:
            stack.append(p)

In [10]:
payload = message.get("payload", {}) or {}
walk_parts(payload)

<generator object walk_parts at 0x124e44220>

In [11]:
payload = message.get("payload", {}) or {}

html_parts = []
for part in walk_parts(message["payload"]):
    if (part.get("mimeType") or "").lower() == "text/html":
        data = (part.get("body") or {}).get("data")
        if data:
            html_parts.append(decode_base64url(data))

len(html_parts)

1

In [12]:
html = html_parts[0] if html_parts else ""
html[:500]

'<!doctype html><html lang="und" dir="auto" xmlns="http://www.w3.org/1999/xhtml" xmlns:v="urn:schemas-microsoft-com:vml" xmlns:o="urn:schemas-microsoft-com:office:office"><head><title></title><!--[if !mso]><!--><meta http-equiv="X-UA-Compatible" content="IE=edge"><!--<![endif]--><meta http-equiv="Content-Type" content="text/html; charset=UTF-8"><meta name="viewport" content="width=device-width,initial-scale=1"><style type="text/css">#outlook a { padding:0; }\r\n      body { margin:0;padding:0;-webk'

## Parse email

### First analysis

In [ ]:
import re

urls = re.findall(r'https?://[^\s"\'>]+', html)

len(urls), urls[:10]

In [ ]:
from urllib.parse import urlparse

domains = [urlparse(u).netloc for u in urls]
set(domains)

In [ ]:
from collections import Counter
Counter(domains).most_common(10)

In [ ]:
import re

urls = re.findall(r'https?://email.s.seek.com.au[^\s"\'>]+', html)

len(urls), urls[:10]

## Second Analysis

In [23]:
from bs4 import BeautifulSoup
from urllib.parse import urlparse
from collections import Counter
import re

soup = BeautifulSoup(html, "html.parser")
links = []
for a in soup.find_all("a", href=True):
    href = a["href"]
    text = a.get_text(" ", strip=True)
    links.append((href, text))

len(links), links[:10]

(34,
 [('https://email.s.seek.com.au/uni/ss/c/u001.Mgc0Pbwk8BklfH5uRvTNRF9V2iyCDLTslpzhbn5pZAjw1w-jrcn9xGn_QNmHKBB6qMtfrzD1FlTWTw7HBld0IRC1hKje6tjVUz5TAjyE61Q/4nv/XnuSGKVwRECDPeNsqW1gUQ/h0/h001.0Bak5Ri2Q4Mu_aTX_34fX7kSAMkRF7lPf8MLV0Flb4o',
   ''),
  ('https://email.s.seek.com.au/uni/ss/c/u001.Mgc0Pbwk8BklfH5uRvTNRBJW9e6gmlmTay3g5FGPg7TRgmHkBzQTH1U2vcclryr8fyBWuqJEti1_a-tQoabzOvddUf592jPbzEnxJZ-5UTc3otouxpZnX7RjviLLwO4KXhFl_4EuuoVEHu7WLUeKlCPtI3ykUfUqsyBGt9KjGY4T44kBrSZDewQVnRjNVe2Y/4nv/XnuSGKVwRECDPeNsqW1gUQ/h1/h001.dHS-siqgrUhUY3cgs6jD7t36E6RdJvk1hs1nuRELlsk',
   'Machine Learning Engineer Preacta Recruitment Sydney NSW'),
  ('https://email.s.seek.com.au/uni/ss/c/u001.Mgc0Pbwk8BklfH5uRvTNRBJW9e6gmlmTay3g5FGPg7R5ZrIrA6nwayqEeLJ9HYR-j3aJoRZBafPhn8u3-CG6t_eVH9nHTjCz0_bZSLNhYECWUoAbABo_blkxCy6_agSpta83CCyh-tLUFr8Ahr6vDxydXksxfDnYwpoNBDlkDw_r1DSecgLR0aFISGN6PHFU/4nv/XnuSGKVwRECDPeNsqW1gUQ/h3/h001.bHMgrzYAL7Qd2mlXtJ8VlePikYhJWm5kX4HgNHiTwj4',
   'Machine Learning Engineer oOh! North Sydney,

In [14]:
def is_job_anchor(a):
    href = a.get("href", "")
    if "email.s.seek.com.au/uni/ss/c/" not in href:
        return False
    div_texts = [d.get_text(" ", strip=True) for d in a.find_all("div")]
    div_texts = [t for t in div_texts if t]
    return len(div_texts) >= 3

job_anchors = [a for a in soup.find_all("a", href=True) if is_job_anchor(a)]
len(job_anchors)

23

In [18]:
a = job_anchors[0]

In [24]:
STATE_RE = re.compile(r"\b(NSW|VIC|QLD|SA|WA|TAS|ACT|NT)\b")
MONEY_RE = re.compile(r"\$[\d,]+")

SALARY_HINT_RE = re.compile(r"\b(salary|super|package|per\s+year|p\.a\.|bonus|incentive|discount)\b", re.I)

def get_style(tag):
    return (tag.get("style") or "").lower()

In [25]:
def candidate_div_texts(a):
    cands = []
    for d in a.find_all("div"):
        txt = d.get_text(" ", strip=True)
        if not txt:
            continue
        # on ignore les gros containers (trop longs) et la duplication grossière
        if len(txt) > 120:
            continue
        cands.append((d, txt))
    return cands


In [19]:
candidate_div_texts(a)

[(<div style="margin:0px auto;max-width:600px;"><table align="center" border="0" cellpadding="0" cellspacing="0" role="presentation" style="width:100%;"><tbody><tr><td style="direction:ltr;font-size:0px;padding:0 0 12px 0;text-align:center;"><!--[if mso | IE]><table role="presentation" border="0" cellpadding="0" cellspacing="0"><tr><td class="cmqym3-outlook rhaa5w-outlook" style="vertical-align:top;width:600px;" ><![endif]--><div class="mj-column-per-100 mj-outlook-group-fix cmqym3 rhaa5w" style="font-size:0px;text-align:left;direction:ltr;display:inline-block;vertical-align:top;width:100%;"><table border="0" cellpadding="0" cellspacing="0" role="presentation" style="border-collapse:separate;" width="100%"><tbody><tr><td style="background-color:#ffffff;border:2px solid #EAECF1;border-radius:16px;vertical-align:top;border-collapse:separate;padding:24px;"><table border="0" cellpadding="0" cellspacing="0" role="presentation" width="100%"><tbody><tr><td align="left" style="font-size:0px;pa

In [26]:
def best_location(cands_texts, title=None, company=None):
    locs = [t for t in cands_texts if STATE_RE.search(t)]
    if not locs:
        return None

    def score(t):
        s = 0
        s += len(t)                  # plus court = mieux
        s += 5 * len(t.split())      # moins de mots = mieux
        if title and title in t:
            s += 500                 # gros malus si concatène le titre
        if company and company in t:
            s += 500                 # gros malus si concatène l'entreprise
        if MONEY_RE.search(t):
            s += 500                 # malus si contient du salaire -> c’est pas une location pure
        return s

    return min(locs, key=score)


In [27]:
def best_salary(cands_texts, location=None):
    money = [t for t in cands_texts if MONEY_RE.search(t)]
    if money:
        # souvent on veut la ligne la plus courte qui contient $
        return min(money, key=len)

    hinted = [t for t in cands_texts if SALARY_HINT_RE.search(t)]
    if not hinted:
        return None
    # éviter de prendre la location si elle contient "per year" etc (rare)
    hinted = [t for t in hinted if t != location]
    return min(hinted, key=len) if hinted else None

In [28]:
def extract_job_from_anchor(a):
    href = a.get("href", "")
    cands = candidate_div_texts(a)
    texts = [txt for _, txt in cands]

    title = None
    company = None
    location = None
    salary = None

    # 1) Title via bold
    for d, txt in cands:
        st = get_style(d)
        if "font-weight:700" in st:
            title = txt
            break

    # 2) Company via grey color
    for d, txt in cands:
        st = get_style(d)
        if "color:#5a6881" in st:
            company = txt
            break

    # 3) Location
    location = best_location(texts, title=title, company=company)

    # 4) Salary via $ OR salary hints (mais on évite de prendre location)
    salary = best_salary(texts, location=location)

    # fallback: si title None, prendre première ligne qui n’est pas company/location/salary
    if not title:
        for _, txt in cands:
            if txt not in {company, location, salary}:
                title = txt
                break

    return {
        "title": title,
        "company": company,
        "location": location,
        "salary": salary,
        "tracking_url": href,
        "debug_lines": [txt for _, txt in cands],
    }


In [29]:
job0 = extract_job_from_anchor(job_anchors[7])
job0


{'title': 'Data Scientist - Natural Perils Pricing',
 'company': 'Suncorp',
 'location': 'Melbourne VIC',
 'salary': 'Competitive Salary + Bens',
 'tracking_url': 'https://email.s.seek.com.au/uni/ss/c/u001.Mgc0Pbwk8BklfH5uRvTNRBJW9e6gmlmTay3g5FGPg7S7sdIYCRQRDsidcwlWEikrkC1eowFWUAxTgBYVRejroorj5hMMyDij83CzMh-9ZnLuNySGILHG_VL7hSR8ULfWTzTrW88KU4mL9f-Eli-QgtLCOfwpv58EIxfsSQ9Df61-KmBauTKlGggxJLh9p7OR/4nv/XnuSGKVwRECDPeNsqW1gUQ/h15/h001.U0S56KanwnmWa3HGvyIeapzo0aqEcpfMeY_CFALrPvI',
 'debug_lines': ['Data Scientist - Natural Perils Pricing Suncorp Melbourne VIC Competitive Salary + Bens',
  'Data Scientist - Natural Perils Pricing Suncorp Melbourne VIC Competitive Salary + Bens',
  'Data Scientist - Natural Perils Pricing',
  'Data Scientist - Natural Perils Pricing',
  'Suncorp',
  'Melbourne VIC',
  'Competitive Salary + Bens']}

In [30]:
location_text = job0.get("location")
print(location_text)
print(location_text.split())
print(" ".join(location_text.split()))

Melbourne VIC
['Melbourne', 'VIC']
Melbourne VIC


In [31]:
import re

STATE_SET = {"NSW","VIC","QLD","SA","WA","TAS","ACT","NT"}
LOC_RE = re.compile(r"^(?:(?P<suburb>.+?),\s*)?(?P<city>.+?)\s+(?P<state>NSW|VIC|QLD|SA|WA|TAS|ACT|NT)$")

def parse_location(location_text: str):
    if not location_text:
        return {"suburb": None, "city": None, "state": None, "location_raw": location_text}

    s = " ".join(location_text.split())
    m = LOC_RE.match(s)
    if not m:
        # fallback: au moins state si présent
        parts = s.split()
        state = parts[-1] if parts and parts[-1] in STATE_SET else None
        return {"suburb": None, "city": None, "state": state, "location_raw": s}

    d = m.groupdict()
    return {
        "suburb": d["suburb"],
        "city": d["city"],
        "state": d["state"],
        "location_raw": s,
    }


In [32]:
RATE_YEAR_RE = re.compile(r"(per\s+year|per\s+annum|p\.?\s*a\.?|pa)\b", re.I)
RATE_DAY_RE  = re.compile(r"(per\s+day|p\.?\s*d\.?|pd)\b", re.I)
RATE_HOUR_RE = re.compile(r"(per\s+hour|p\.?\s*h\.?|ph|hourly)\b", re.I)

def detect_rate_type(s: str):
    if RATE_HOUR_RE.search(s):
        return "hour"
    if RATE_DAY_RE.search(s):
        return "day"
    if RATE_YEAR_RE.search(s):
        return "year"
    return None

In [ ]:
MONEY_NUM_RE = re.compile(r"\$\s*([0-9]+(?:,[0-9]{3})*(?:\.[0-9]+)?)")

def parse_salary(salary_text: str):
    if not salary_text:
        return {
            "salary_min": None, "salary_max": None,
            "salary_period": None,
            "salary_text": None
        }

    s = " ".join(salary_text.split())
    period = detect_rate_type(s)

    if "$" not in s:
        return {
            "salary_min": None, "salary_max": None,
            "salary_period": period,
            "salary_text": s
        }

    nums = [float(n.replace(",", "")) for n in MONEY_NUM_RE.findall(s)]
    if not nums:
        return {
            "salary_min": None, "salary_max": None,
            "salary_period": period,
            "salary_text": s
        }
    
    if period is None and max(nums) >= 1000:
        period = "year"

    if len(nums) >= 2:
        lo, hi = nums[0], nums[1]
        return {
            "salary_min": min(lo, hi),
            "salary_max": max(lo, hi),
            "salary_period": period,
            "salary_text": s
        }

    return {
        "salary_min": nums[0],
        "salary_max": nums[0],
        "salary_period": period,
        "salary_text": s
    }

In [34]:
def normalize_job(job: dict) -> dict:
    # 1) location -> suburb/city/state/location_raw
    loc = parse_location(job.get("location"))
    job.pop("location", None)            # supprime l'ancien champ
    job.update(loc)                      # ajoute suburb/city/state/location_raw

    # 2) salary -> salary_min/max/text
    sal = parse_salary(job.get("salary"))
    job.pop("salary", None)              # supprime l'ancien champ
    job.update(sal)                      # ajoute salary_min/max/salary_text

    return job

def reorder_job(job: dict) -> dict:
    ordered_keys = [
        "title", "company",
        "suburb", "city", "state", "location_raw",
        "salary_min", "salary_max", "salary_period", "salary_text",
        "tracking_url"
    ]
    # ajoute le reste à la fin (debug_lines etc.)
    rest = [k for k in job.keys() if k not in ordered_keys]
    return {k: job.get(k) for k in ordered_keys + rest}

In [40]:
j={}
j["title"].update("d")

KeyError: 'title'

In [35]:
job = extract_job_from_anchor(job_anchors[5])
job = reorder_job(normalize_job(job))
job

{'title': 'ML/AI Specialist',
 'company': 'UpperGround by Hudson',
 'suburb': None,
 'city': 'Brisbane',
 'state': 'QLD',
 'location_raw': 'Brisbane QLD',
 'salary_min': None,
 'salary_max': None,
 'salary_period': 'hour',
 'salary_text': 'AUD 118 - 121 per hour, + super',
 'tracking_url': 'https://email.s.seek.com.au/uni/ss/c/u001.Mgc0Pbwk8BklfH5uRvTNRBJW9e6gmlmTay3g5FGPg7TowWhceRd_IYrFJLFkqXHPY9IYQkBxXaRaHBctgBgYhpN3OkJ4I8hqTYccDX-YnSpVMLTokOT0rFAZC-aJI2cNTyxkYRzedzZvF7vtknv3VJ_ibMmhD51k5cwU2DwBRAywx4svm3hmergZ_R8pR4XX/4nv/XnuSGKVwRECDPeNsqW1gUQ/h11/h001.cd4gKcDFJlBzvT5A9_FD1HWRa7kBupeHT13qgGAUnUw',
 'debug_lines': ['ML/AI Specialist UpperGround by Hudson Brisbane QLD AUD 118 - 121 per hour, + super',
  'ML/AI Specialist UpperGround by Hudson Brisbane QLD AUD 118 - 121 per hour, + super',
  'ML/AI Specialist',
  'ML/AI Specialist',
  'UpperGround by Hudson',
  'Brisbane QLD',
  'AUD 118 - 121 per hour, + super']}

In [ ]:
job = extract_job_from_anchor(job_anchors[13])
job = reorder_job(normalize_job(job))
job

In [ ]:
job = extract_job_from_anchor(job_anchors[1])
job = reorder_job(normalize_job(job))
job

# boT

In [ ]:
import re
from urllib.parse import urlparse

PAT = re.compile(
    r"^https://email\.s\.seek\.com\.au/uni/ss/c/(?P<payload>.+?)/(?P<batch>4n[a-z])/(?P<search_id>[^/]+)/h(?P<pos>\d+)/h001\.(?P<tail>.+)$",
    re.IGNORECASE
)

rows = []
for href, text in links:
    m = PAT.match(href)
    if not m:
        continue
    d = m.groupdict()
    d["href"] = href
    d["text"] = text
    d["pos"] = int(d["pos"])
    rows.append(d)

len(rows), rows